#     Baseline Training and Testing using the unpreprocessed dataset

In [1]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from statistics import mean

from sklearn import linear_model, tree, svm
from sklearn.datasets import load_digits
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_roc_curve
from sklearn.datasets import load_wine
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

%matplotlib inline

In [2]:
df = pd.read_csv('../../data/infdata_stage3.csv')

In [3]:
df.head()

,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch
0,0,0.156250,0.000,0.791582,0.850586,0.269199,0.393398,0.029297,0.080078,0.023438,...,0,1,0,0,0,0,0,0,1,0
1,1,0.046875,0.125,0.459277,0.404687,0.322188,0.068867,0.189453,0.000000,0.283203,...,0,1,0,0,0,0,1,0,0,0
2,1,0.015625,0.000,0.450000,0.427539,0.250000,0.200000,0.148438,0.093750,0.000000,...,0,0,0,1,0,0,0,0,0,1
3,0,0.000000,0.000,0.420000,0.495000,0.330000,0.360000,0.500000,1.000000,0.000000,...,0,1,0,0,0,0,1,0,0,0
4,0,0.015625,0.000,0.450000,0.427539,0.250000,0.200000,0.148438,0.093750,0.000000,...,0,1,0,0,0,0,0,0,1,0


In [4]:
#splitting up data into training/testing
y = df['Winner']
X = df.drop(columns = 'Winner')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=43)

In [5]:
#Creating Model Instances
logistic_regression_model = LogisticRegression(max_iter=90000)
adaboost_model = AdaBoostClassifier(n_estimators=500, random_state=0)#changed from 100 to 500
ann_model = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 5, 5, 2), random_state=1, max_iter=650, learning_rate='adaptive')
dtree_model = tree.DecisionTreeClassifier()
#naivebayesgaussian_model = GaussianNB()
randforest_model = RandomForestClassifier(max_depth=2, random_state=0)
svm_model = svm.SVC(decision_function_shape='ovo')
xgboost_model = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=3)

In [6]:
#arrays to hold 
scores_logistic_reg = []
scores_adaboost = []
#scores_naiveBayes = []
scores_svm = []
scores_dt = []
scores_randforest = []
scores_ann = []
scores_xgboost = []

In [7]:
'''
This function trains/test and returns the accuracy of that iteration
'''
def get_score(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    return model.score(x_test, y_test)

In [8]:
#N-Fold Cross validation will be performed
kf = KFold(n_splits=8) #changed from 4 - 7 - 20 -3

#     Training

In [9]:
#generating and appending scores
counter= 0
for train_index, test_index in kf.split(X_train, y_train):
    print("counter: " + str(counter))
    counter+=1
    xtrain = X_train.iloc[min(train_index):max(train_index)]
    xtest = X_train.iloc[min(test_index):max(test_index)]
    ytrain = y_train.iloc[min(train_index):max(train_index)]
    ytest = y_train.iloc[min(test_index):max(test_index)]
    
    scores_xgboost.append(get_score(xgboost_model, xtrain, xtest, ytrain, ytest))
    scores_logistic_reg.append(get_score(logistic_regression_model, xtrain, xtest, ytrain, ytest))
    scores_ann.append(get_score(ann_model, xtrain, xtest, ytrain, ytest))
#     scores_naiveBayes.append(get_score(naivebayesgaussian_model, xtrain, xtest, ytrain, ytest))
    scores_svm.append(get_score(svm_model, xtrain, xtest, ytrain, ytest))
    scores_adaboost.append(get_score(adaboost_model, xtrain, xtest, ytrain, ytest))
    scores_dt.append(get_score(dtree_model, xtrain, xtest, ytrain, ytest))
    scores_randforest.append(get_score(randforest_model, xtrain, xtest, ytrain, ytest))


counter: 0


ValueError: could not convert string to float: 'Jun 22, 1983'

In [10]:
bestScore = -1
best_model = None
if mean(scores_logistic_reg) > bestScore:
    best_model = logistic_regression_model
    bestScore = mean(scores_logistic_reg)
print("Log Reg Mean: " + str(mean(scores_logistic_reg)))

# if mean(scores_naiveBayes) > bestScore:
#     best_model = naivebayesgaussian_model
#     bestScore = mean(scores_naiveBayes)
# print("NaiveBayes Mean: " + str(mean(scores_naiveBayes)))

if mean(scores_xgboost) > bestScore:
    best_model = xgboost_model
    bestScore = mean(scores_xgboost)
print("XGBoost Mean: " + str(mean(scores_xgboost)))

if mean(scores_svm) > bestScore:
    best_model = svm_model
    bestScore = mean(scores_svm)
print("SVM Mean: " + str(mean(scores_svm)))

if mean(scores_adaboost) > bestScore:
    best_model = adaboost_model
    bestScore = mean(scores_adaboost)
print("Adaboost Mean: " + str(mean(scores_adaboost)))

if mean(scores_dt) > bestScore:
    best_model = dtree_model
    bestScore = mean(scores_dt)
print("DT Mean: " + str(mean(scores_dt)))

if mean(scores_randforest) > bestScore:
    best_model = randforest_model
    bestScore = mean(scores_randforest)
print("RandForest Mean: " + str(mean(scores_randforest)))

if mean(scores_ann) > bestScore:
    best_model = ann_model
    bestScore = mean(scores_ann)
print("ANN Mean: " + str(mean(scores_ann)))

print("")
print("Best Model: " +str(best_model) + " with a score of: " + str(bestScore) )

Log Reg Mean: 0.6920158959537572
XGBoost Mean: 0.747471098265896
SVM Mean: 0.5603323699421965
Adaboost Mean: 0.7548771676300579
DT Mean: 0.8973988439306358
RandForest Mean: 0.6591401734104047
ANN Mean: 0.5037933526011561

Best Model: DecisionTreeClassifier() with a score of: 0.8973988439306358


# Testing

In [11]:
#decision tree
y_preds = dtree_model.predict(X_test)
accuracy_score(y_test, y_preds)

0.6198630136986302

In [12]:
#random forest
y_preds = randforest_model.predict(X_test)
accuracy_score(y_test, y_preds)

0.6472602739726028

In [13]:
#adaboost
y_preds = adaboost_model.predict(X_test)
accuracy_score(y_test, y_preds)

0.6301369863013698

In [14]:
#ann
y_preds = ann_model.predict(X_test)
accuracy_score(y_test, y_preds)

0.4486301369863014

In [15]:
#support vector machine
y_preds = svm_model.predict(X_test)
accuracy_score(y_test, y_preds)

0.5376712328767124

In [16]:
# svc_disp = plot_roc_curve(svm_model, X_test, y_test)
# plt.show()

In [17]:
#support vector machine
y_preds = logistic_regression_model.predict(X_test)
accuracy_score(y_test, y_preds)

0.6883561643835616

In [18]:
#xgboost
y_preds = xgboost_model.predict(X_test)
accuracy_score(y_test, y_preds)

0.6746575342465754

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_preds)

array([[ 95,  36],
       [ 59, 102]])